In [5]:
import json 
import os 
import matplotlib.pyplot as plt
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

data={"Le Vésinet": "le_vesinet.csv", "Montesson": "montesson.csv", "Chatou": "chatou.csv" }

nom_ville = "Chatou"
df = pd.read_csv(data[nom_ville],sep=';')

lon = []
lat = []

for i in df.iloc[:,0]:
    li = i.split(",")
    lat.append(float(li[0]))
    lon.append(float(li[1]))

df.insert(loc=1, column='lon', value=lon)
df.insert(loc=1, column='lat', value=lat)

df['index'] = df.index

fig = px.scatter_mapbox(df, 
                        lat="lat", 
                        lon="lon", 
                        hover_name="index",
                        hover_data={"index":False,
                                    "Departementale":True,
                                    "M12":True,
                                    "Sas velo":True,
                                    "lon":False,
                                    "lat":False},
                        zoom=13) 

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show() 

In [6]:
nb_feux = 0
nb_feux_depa = 0

nb_m12 = 0
nb_sas = 0

nb_m12_depa = 0
nb_sas_depa = 0

for k in df["index"]:
    nb_feux+=1
    nb_m12 += (df.iloc[k,:]["M12"] == "Oui")
    nb_sas += (df.iloc[k,:]["Sas velo"] == "Oui")
    nb_feux_depa += (df.iloc[k,:]["Departementale"] == "Oui")
    nb_m12_depa += (df.iloc[k,:]["Departementale"] == "Oui") and (df.iloc[k,:]["M12"] == "Oui")
    nb_sas_depa += (df.iloc[k,:]["Departementale"] == "Oui") and (df.iloc[k,:]["Sas velo"] == "Oui")

fig = make_subplots(
    rows=3, cols=1,
    vertical_spacing=0.04,
    specs=[[{"type": "table"}],
           [{"type": "table"}] ,
           [{"type": "table"}] ],
    subplot_titles=(nom_ville +" : ensemble de la voirie", nom_ville + " : départementales", nom_ville + " : départementales exclues")
)

fig1 = go.Table(header=dict(values=["Feux tricolores","Panneaux M12", "Sas vélo", "Panneaux M12/feux [%]", "Sas vélo/feux [%]"]),
                 cells=dict(values=[[nb_feux],[nb_m12],[nb_sas], [round(100*nb_m12/nb_feux,1)], [round(100*nb_sas/nb_feux,1)]]))

fig2 = go.Table(header=dict(values=["Feux tricolores","Panneaux M12", "Sas vélo", "Panneaux M12/feux [%]", "Sas vélo/feux [%]"]),
                 cells=dict(values=[[nb_feux_depa],[nb_m12_depa],[nb_sas_depa], [round(100*(nb_m12_depa)/(nb_feux_depa),1)], [round(100*(nb_sas_depa)/(nb_feux_depa),1)]]))
                     
fig3 = go.Table(header=dict(values=["Feux tricolores","Panneaux M12", "Sas vélo", "Panneaux M12/feux [%]", "Sas vélo/feux [%]"]),
                 cells=dict(values=[[nb_feux-nb_feux_depa],[nb_m12 - nb_m12_depa],[nb_sas-nb_sas_depa], [round(100*(nb_m12-nb_m12_depa)/(nb_feux-nb_feux_depa),1)], [round(100*(nb_sas-nb_sas_depa)/(nb_feux-nb_feux_depa),1)]]))
                     

fig.add_trace(fig1, row=1, col=1)
fig.add_trace(fig2, row=2, col=1)
fig.add_trace(fig3, row=3, col=1)

fig.update_layout(height=600, width=1600)

fig.show()